## This is an optional notebook to assist troubleshooting of cloudwatch logs primarily for Tool use verification with GenAI Observability

In [ ]:
import boto3
from datetime import datetime, timedelta

def extract_logs_for_session(session_id, log_group_name='/aws/bedrock/agentcore', days_back=1):
    logs_client = boto3.client('logs')
    
    end_time = datetime.utcnow()
    start_time = end_time - timedelta(days=days_back)
    
    response = logs_client.filter_log_events(
        logGroupName=log_group_name,
        filterPattern=session_id
    )
    
    return [event['message'] for event in response.get('events', [])]

# Usage
session_id = "eval-session-f9400df5-6198-4f19-a491-fa1e8fcafa4f"
logs = extract_logs_for_session(session_id, log_group_name='/aws/bedrock-agentcore/runtimes/citysearch-583XaH96wT-DEFAULT')
logs

/var/folders/x7/t3b9975n6_z57y4hq5d6yhfc0000gr/T/ipykernel_2512/1805218374.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_time = datetime.utcnow()


['{"resource":{"attributes":{"deployment.environment.name":"bedrock-agentcore:default","aws.local.service":"citysearch.DEFAULT","service.name":"citysearch.DEFAULT","cloud.region":"us-east-1","aws.log.stream.names":"otel-rt-logs","telemetry.sdk.name":"opentelemetry","aws.service.type":"gen_ai_agent","telemetry.sdk.language":"python","cloud.provider":"aws","cloud.resource_id":"arn:aws:bedrock-agentcore:us-east-1:146666888814:runtime/citysearch-583XaH96wT/runtime-endpoint/DEFAULT:DEFAULT","aws.log.group.names":"/aws/bedrock-agentcore/runtimes/citysearch-583XaH96wT-DEFAULT","telemetry.sdk.version":"1.33.1","cloud.platform":"aws_bedrock_agentcore","telemetry.auto.version":"0.12.0-aws"}},"scope":{"name":"opentelemetry.instrumentation.botocore.bedrock-runtime","schemaUrl":"https://opentelemetry.io/schemas/1.30.0"},"timeUnixNano":1757858144548182791,"observedTimeUnixNano":1757858144548200496,"severityNumber":9,"severityText":"","body":{"message":{"content":[{"text":"<thinking> It seems that th

In [13]:
import json
cwlog_name='./observability_samples_cwlogs/cloudwatch_logs_sept18.jsonl'
def write_logs_as_jsonl(logs, filename):
    with open(filename, 'w') as f:
        for log in logs:
            f.write(json.dumps(log) + '\n')
write_logs_as_jsonl(logs,cwlog_name )

In [10]:
import json
json.loads(logs[2])

{'resource': {'attributes': {'deployment.environment.name': 'bedrock-agentcore:default',
   'aws.local.service': 'citysearch.DEFAULT',
   'service.name': 'citysearch.DEFAULT',
   'cloud.region': 'us-east-1',
   'aws.log.stream.names': 'otel-rt-logs',
   'telemetry.sdk.name': 'opentelemetry',
   'aws.service.type': 'gen_ai_agent',
   'telemetry.sdk.language': 'python',
   'cloud.provider': 'aws',
   'cloud.resource_id': 'arn:aws:bedrock-agentcore:us-east-1:146666888814:runtime/citysearch-583XaH96wT/runtime-endpoint/DEFAULT:DEFAULT',
   'aws.log.group.names': '/aws/bedrock-agentcore/runtimes/citysearch-583XaH96wT-DEFAULT',
   'telemetry.sdk.version': '1.33.1',
   'cloud.platform': 'aws_bedrock_agentcore',
   'telemetry.auto.version': '0.12.0-aws'}},
 'scope': {'name': 'strands.telemetry.tracer'},
 'timeUnixNano': 1757858144585227778,
 'observedTimeUnixNano': 1757858144585362336,
 'severityNumber': 9,
 'severityText': '',
 'body': {'output': {'messages': [{'content': {'content': '[{"text"

In [15]:
# read jsonl file
with open(cwlog_name, 'r') as f:
    cwlogs = [json.loads(line) for line in f]
    print(cwlogs[1])


{"resource":{"attributes":{"deployment.environment.name":"bedrock-agentcore:default","aws.local.service":"citysearch.DEFAULT","service.name":"citysearch.DEFAULT","cloud.region":"us-east-1","aws.log.stream.names":"otel-rt-logs","telemetry.sdk.name":"opentelemetry","aws.service.type":"gen_ai_agent","telemetry.sdk.language":"python","cloud.provider":"aws","cloud.resource_id":"arn:aws:bedrock-agentcore:us-east-1:146666888814:runtime/citysearch-583XaH96wT/runtime-endpoint/DEFAULT:DEFAULT","aws.log.group.names":"/aws/bedrock-agentcore/runtimes/citysearch-583XaH96wT-DEFAULT","telemetry.sdk.version":"1.33.1","cloud.platform":"aws_bedrock_agentcore","telemetry.auto.version":"0.12.0-aws"}},"scope":{"name":"strands.telemetry.tracer"},"timeUnixNano":1757858144548792439,"observedTimeUnixNano":1757858144549011918,"severityNumber":9,"severityText":"","body":{"output":{"messages":[{"content":{"content":"[{\"text\": \"<thinking> To determine the population and area of New York City, I will first perfor

In [18]:
import json

def extract_tooluse_from_log(log_message):
    tools = []
    
    try:
        log_data = json.loads(log_message)
        
        # Navigate to output messages
        messages = log_data.get('body', {}).get('output', {}).get('messages', [])
        
        for message in messages:
            content = message.get('content', {}).get('content', '')
            
            # Parse the content JSON string
            if content:
                content_data = json.loads(content)
                # Extract toolUse from each item
                for item in content_data:
                    if 'toolUse' in item:
                        tool_name = item['toolUse']['name']
                        tools.append(tool_name)
                        
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error parsing log: {e}")
    
    return tools

def extract_tools_from_all_logs(logs_list):
    all_tools = []
    for log in logs_list:
        tools = extract_tooluse_from_log(log)
        all_tools.extend(tools)
    return all_tools

# Example usage
all_tools = extract_tools_from_all_logs(cwlogs)
print(f"All tools found: {list(set(all_tools))}")

All tools found: ['web_search']


In [ ]:
from typing import Dict, List, Any, Optional
import time
import uuid

AGENT_NAME = "citysearch"
AGENT_ENDPOINT = "arn:aws:bedrock-agentcore:us-east-1:146666888814:runtime/citysearch-583XaH96wT"
agentcore_client = boto3.client('bedrock-agentcore', region_name='us-east-1')

async def invoke_agent(query: str) -> Dict[str, Any]:
    """Invoke the agent using AgentCore Runtime"""
    start_time = time.time()
    
    try:
        payload = json.dumps({"prompt": query})
        session_id = f"eval-session-{uuid.uuid4()}"
        print(f"Session ID: {session_id}")
        response = agentcore_client.invoke_agent_runtime(
            agentRuntimeArn=AGENT_ENDPOINT,
            runtimeSessionId=session_id,
            payload=payload,
            qualifier="DEFAULT"
        )
        
        print("AgentCore Response keys:", list(response.keys()))
        print("AgentCore Response Traceid :", response.get('traceId', ''))
        # Extract response text from StreamingBody
        response_text = ""
        if isinstance(response, dict) and 'response' in response:
            streaming_body = response['response']
            if hasattr(streaming_body, 'read'):
                response_text = streaming_body.read().decode('utf-8')
                print(f"Extracted response text: {response_text}")
            else:
                response_text = str(streaming_body)
        else:
            response_text = str(response)
        return response
    except Exception as e:
        error_msg = f"Error invoking agent: {str(e)}"
        print(error_msg)        

In [37]:
agent_result = await invoke_agent("How many residents in cypress orange county?")

Session ID: eval-session-78c4db35-7e94-4fe6-b6fe-b45bc6439cbd
AgentCore Response keys: ['ResponseMetadata', 'runtimeSessionId', 'traceId', 'baggage', 'contentType', 'statusCode', 'response']
AgentCore Response Traceid : Root=1-68cccdf6-08060d654af50eb30f9f8cf2;Self=1-68cccdf6-716fea7c4f6effdb32d15948
Extracted response text: "<thinking> Based on the web search results, I found the most recent data about the population of Cypress, Orange County. According to the sources, the population of Cypress as of the 2020 census was 50,151. However, the most recent estimate from 2025 indicates a decrease in population, with an estimated population of 47,938. This represents a decline of approximately 4.21% since the last census. Now, I will provide this information in the requested format.</thinking>\n\nThe population of Cypress, Orange County, as of 2025 is estimated to be **47,938**. This figure reflects a decrease from the 2020 census population of **50,151**."


In [39]:
agent_result

{'ResponseMetadata': {'RequestId': '1d692087-1d0f-4242-b59b-9a4e2dc87c7c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 19 Sep 2025 03:28:57 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1d692087-1d0f-4242-b59b-9a4e2dc87c7c',
   'baggage': 'Self=1-68cccdf6-716fea7c4f6effdb32d15948,session.id=eval-session-78c4db35-7e94-4fe6-b6fe-b45bc6439cbd',
   'x-amzn-bedrock-agentcore-runtime-session-id': 'eval-session-78c4db35-7e94-4fe6-b6fe-b45bc6439cbd',
   'x-amzn-trace-id': 'Root=1-68cccdf6-08060d654af50eb30f9f8cf2;Self=1-68cccdf6-716fea7c4f6effdb32d15948'},
  'RetryAttempts': 0},
 'runtimeSessionId': 'eval-session-78c4db35-7e94-4fe6-b6fe-b45bc6439cbd',
 'traceId': 'Root=1-68cccdf6-08060d654af50eb30f9f8cf2;Self=1-68cccdf6-716fea7c4f6effdb32d15948',
 'baggage': 'Self=1-68cccdf6-716fea7c4f6effdb32d15948,session.id=eval-session-78c4db35-7e94-4fe6-b6fe-b45bc6439cbd',
 'contentType': 'application/jso

In [48]:
import json

response_dict = agent_result['response']
content = response_dict.read().decode('utf-8')

In [ ]:
{'ResponseMetadata': {'RequestId': 'd807784c-ed7c-406f-8ee8-fe872f90cca6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 19 Sep 2025 03:26:43 GMT', 'content-type': 'application/json', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': 'd807784c-ed7c-406f-8ee8-fe872f90cca6', 'baggage': 'Self=1-68cccd71-54c1dd5d3da56d131435093e,session.id=eval-session-5b887a5d-0c64-433c-ae75-1a67983ed3d2', 'x-amzn-bedrock-agentcore-runtime-session-id': 'eval-session-5b887a5d-0c64-433c-ae75-1a67983ed3d2', 'x-amzn-trace-id': 'Root=1-68cccd71-092260e9268c77523c6b6179;Self=1-68cccd71-54c1dd5d3da56d131435093e'}, 'RetryAttempts': 0}, 'runtimeSessionId': 'eval-session-5b887a5d-0c64-433c-ae75-1a67983ed3d2', 'traceId': 'Root=1-68cccd71-092260e9268c77523c6b6179;Self=1-68cccd71-54c1dd5d3da56d131435093e', 'baggage': 'Self=1-68cccd71-54c1dd5d3da56d131435093e,session.id=eval-session-5b887a5d-0c64-433c-ae75-1a67983ed3d2', 'contentType': 'application/json', 'statusCode': 200, 'response': <botocore.response.StreamingBody object at 0x11782d7b0>}
